# Homework Assignment 3

You may use whatever IDEs Platforms / text editors you like, but you mustsubmit your responses and source code files (Jupyter notebooks are considered as such) on iCollege.

Note: Please refer to the Jupyter notebooks created in class. They provide useful hints for solving the following problems. In the submitted Jupyter notebook(s), your code cells’ output (if any) should be visible without executing the notebook

You may use whatever IDEs Platforms / text editors you like, but you mustsubmit your responses and source code files (Jupyter notebooks are considered as such) on iCollege.

Note: Please refer to the Jupyter notebooks created in class. They provide useful hints for solving the following problems. In the submitted Jupyter notebook(s), your code cells’ output (if any) should be visible without executing the notebook

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import regex as re
from tqdm.auto import tqdm
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


(a) (1 point) Prove that the database file has been properly loaded by showing (output requested) that the table “wiki_articles_extended” exists and the pandas dataframe “df” contains the expected columns “title”, “text”, “name”, “url”, and “nouns” (along with many other columns) as well as 60 rows for the respective Wikipedia texts/documents. 

In [4]:
con = sqlite3.connect("wiki-articles-extended.db")

df = pd.read_sql_query("SELECT * from wiki_articles_extended", con)
#con.close()
df 

,index,title,text,name,url,datePublished,headline,nr_tokens,nr_characters,nouns,adjectives,verbs,lemmas,nav,entities,noun_chunks,no_tokens,no_sentences,no_noun_chunks
0,0,Abuse case,"From Wikipedia, the free encyclopedia\n\n\nAbu...",Abuse case,https://en.wikipedia.org/wiki/Abuse_case,2010-03-19T13:30:06Z,specification model for security requirements ...,477,2607,Abuse|case|specification|model|security|requir...,complete|more|harmful#just|coherent#instead|ac...,be|use#be#be|introduce|work#define|be|be#can|d...,\n\n\n|Abuse|case| |be|a|specification|model|f...,Abuse|case|be|specification|model|security|req...,John McDermott|Chris Fox|1999|Computer Science...,\n\n\nAbuse case|a specification model|securit...,433.0,21.0,87.0
1,1,Application security,"From Wikipedia, the free encyclopedia\n\n\nMea...",Application security,https://en.wikipedia.org/wiki/Application_secu...,2005-08-29T21:54:46Z,measures taken to improve the security of an a...,1678,9360,measure|security|application|part|series|onInf...,related|automotive|electronic|mobile|adware|ad...,take|improve|script|object|scrape|spoof|phishe...,\n\n\n|measure|take|to|improve|the|security|of...,measure|take|improve|security|application|part...,onInformation|Digital|Crimeware\nCross|SIEM|Ap...,\n\n\nMeasures|the security|an application|Par...,1500.0,81.0,229.0
2,2,Asset (computer security),"From Wikipedia, the free encyclopedia\n\n\nDat...",Asset (computer security),https://en.wikipedia.org/wiki/Asset_(computer_...,2010-11-21T12:40:30Z,"any data, device, or other component of the en...",454,2570,Data|device|component|environment|use|Asset|di...,other|other#other#generally|e.g.|e.g.|critical...,compute|see#be|support|relate#include#be|ensur...,"\n\n\n|Data|,|device|,|or|other|component|of|a...",Data|device|other|component|compute|environmen...,CIA|CIA#ISO/IEC 13335-1:2004|1#2012-02-29#2010...,\n\n\nData|device|other component|a computing ...,374.0,16.0,76.0
3,3,Attack tree,"From Wikipedia, the free encyclopedia\n\n\nCon...",Attack tree,https://en.wikipedia.org/wiki/Attack_tree,2006-03-10T05:06:02Z,"conceptual diagrams showing how an asset, or t...",1852,9567,diagram|asset|target|article|citation|verifica...,conceptual|additional#reliable#unsourced#conce...,show|might|be|attack|need#help|improve|add#may...,"\n\n\n|conceptual|diagram|show|how|an|asset|,|...",conceptual|diagram|show|asset|target|might|be|...,JSTOR|April 2012#Bruce Schneier|the National S...,\n\n\nConceptual diagrams|an asset|target|This...,1545.0,74.0,261.0
4,4,Automated threat,"From Wikipedia, the free encyclopedia\n\n\nPar...",Automated threat,https://en.wikipedia.org/wiki/Automated_threat,2016-09-18T10:40:20Z,#empty#,648,3522,Part|series|onAutomation|Automation|Banking|bu...,general|console|clean|reasoning|semi|-|automat...,guide|drive|automate|be|characterise|automate#...,\n\n\n|Part|of|a|series|onAutomation|\n|Automa...,Part|series|onAutomation|Automation|general|Ba...,IEEE Robotics#CAPTCHA Bypass|Card Cracking\nId...,Part|a series|general\n\nBanking\nBuilding\nHo...,591.0,15.0,81.0
5,5,Browser security,"From Wikipedia, the free encyclopedia\n\n\nApp...",Browser security,https://en.wikipedia.org/wiki/Browser_security,2012-03-05T13:20:27Z,application of internet security to web browsers,4565,22826,Application|internet|security|web|browser|Brow...,networked#often|sometimes|cross|-|site#more|fo...,be|protect#use#can|be|breach|be|breach|be|read...,\n\n\n|Application|of|internet|security|to|web...,Application|internet|security|web|browser|Brow...,JavaScript#one#Improper Access Control|CWE-284...,Application|internet security|web browsers|\n\...,3608.0,232.0,522.0
6,6,Buffer overflow,"From Wikipedia, the free encyclopedia\n\n\nAno...",Buffer overflow,https://en.wikipedia.org/wiki/Buffer_overflow,2001-10-05T22:29:45Z,anomaly in computer security and programming t...,6958,35057,Anomaly|computer|security|programming|visualiz...,too|large|adjacent#aside|often#often|malformed...,be|write|be|fit|overflow|be|write|overrun|ov

(b) (2 points) Extract the most common 7500 nouns from all the 60 Wikipedia texts. These tokens must be in lower case and have a length of at least 1. Also, they cannot be stop words. For this purpose, please use the code fragments provided. Afterwards, the object “nouns” will map from a token to its overall frequency. The list “voc” contains the strings of tokens extracted, sorted in descending order of their frequency of occurrence (the token at the largest index position 7499 will be the least frequent one). Thus, this list can be used to find an index for a specific token. For instance, the index of token “information” in this list could be 3, found by calling voc.index("information") . Show that the list “voc” contains 7500 tokens and that there are no 
tokens with a frequency of 0 in it (the last element in it must appear at least 1 time).

In [5]:
def single_words(df, field):
    return [w.lower() for words in df[field] for w in re.split(r'\||\#', words) if len(w)>1]

In [6]:
nouns = Counter(single_words(df, "nouns"))
for w in STOP_WORDS:
    nouns[w] = 0
    
voc = [w for w, freq in nouns.most_common(7500)]
#voc.index("information")

assert len(voc) == 7500   # assert is showing that the list “voc” contains 7500 tokens
assert min(nouns[word] for word in voc) > 0 # assert shows that there are no tokens with a frequency of 0 in it (the last element in it must appear at least 1 time).

### Tasks 1a and 1b

(c) (15 points) Create a co-occurrence matrix (named e.g. “coocc_matrix”) using a 2-dimensional Numpy array that has a shape of (7500x7500). Each element in this matrix should be initialized to 0 and will count how often a token i appears together with a token j in df[“nouns”] using a cooccurrence window of 5. The tokens to be extracted (in lower case, no stop words) from df[“nouns”] must appear in the previously obtained list “voc”. For instance, the element at (3, 0) could be accessed by “coocc_matrix[0][3]”. As an undirected co-occurrence graph is generated this way, the value of “coocc_matrix[0][3]” must match that of “coocc_matrix[3][0]”. The mentioned indices 0 and 3 can be obtained by calling voc.index("information") and 
voc.index("security"). 

Note: Co-occurrences of a token with itself in the co-occurrence window should be counted.

In [7]:
from collections import defaultdict
c = defaultdict(lambda: Counter())


coocc_matrix = np.zeros((len(voc), len(voc))) # numpy np.zeros is initializing co-occurrence matrix with zeros with shape of (7500x7500)

window = 5 # The initialization of window size to 5
skip = (window - 1) // 2
for doc in tqdm(df["nouns"]):
   
    tokens = [w for w in re.split(r'\||\#', doc) if w not in STOP_WORDS]  # The removal of stop words
    for i, w in enumerate(tokens):
        if w in voc:
            for j in range(max(0, i-skip), i):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1
            for j in range(i+1, min(i+1+skip, len(tokens))):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1


for key, counter in c.items(): # The statement Iterating over each key-value pair in the dictionary
    i = voc.index(key)
    for k, v in counter.items():
        j = voc.index(k)
        coocc_matrix[i][j] = v
        coocc_matrix[j][i] = v  # Since it's an undirected graph, assign the same value to both positions


print(coocc_matrix) # Print the resulting co-occurrence matrix        


print(coocc_matrix[0][3]) 
print(coocc_matrix[3][0]) 
print(coocc_matrix[voc.index("security")][voc.index("information")]) 
print(coocc_matrix[voc.index("information")][voc.index("security")]) 

  0%|          | 0/60 [00:00<?, ?it/s]

[[436. 189. 178. ...   0.   0.   0.]
 [189. 166.  99. ...   0.   0.   0.]
 [178.  99. 176. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   1.   1.]
 [  0.   0.   0. ...   1.   0.   1.]
 [  0.   0.   0. ...   1.   1.   0.]]
322.0
322.0
322.0
322.0


(d) (3 points) For all indices i and j, prove that the elements “coocc_matrix[i][j]” and “coocc_matrix[j][i]” have the same values.

In [8]:
print(coocc_matrix[0][3]) 
print(coocc_matrix[3][0]) 
assert coocc_matrix[0][3] == coocc_matrix[3][0] # assert will raises an exception if the condition is not met 
                                                # Because this assert is not raising an execption hence “coocc_matrix[i][j]” and “coocc_matrix[j][i]” have the same values.


print(coocc_matrix[voc.index("security")][voc.index("information")]) 
print(coocc_matrix[voc.index("information")][voc.index("security")]) 
assert coocc_matrix[voc.index("security")][voc.index("information")] == coocc_matrix[voc.index("information")][voc.index("security")]

322.0
322.0
322.0
322.0


# Co-occurrence analysis (main homework)


(2) (7 points) Extend the Jupyter notebook (in Python) created in 1) by providing solutions to the following tasks:

(a) Using the basic PageRank algorithm (not the extended variant) applied to a stochastic matrix derived from the co-occurrence matrix obtained in 1c), output the ten most important 
keywords/tokens of “voc” sorted in descending order by their PageRank values.

Note: For this purpose, you can use the function “pagerank()” from the Python-example under https://en.wikipedia.org/wiki/PageRank , which computes the PageRank scores using the power 
method described in this Wikipedia-article. In order to be able to do so, you must first transform the obtained co-occurrence matrix into a stochastic matrix by e.g. calling 
stochastic_matrix = coocc_matrix / (coocc_matrix.sum(axis=0, keepdims=True)+0.001). Then, the mentioned function “pagerank” can be called by v = pagerank(stochastic_matrix) .

In [10]:
stochastic_matrix = coocc_matrix / (coocc_matrix.sum(axis=0, keepdims=True)+0.001) # Compute the stochastic matrix

def pagerank(M, num_iterations=100, d=0.85): # Implement the PageRank algorithm
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    M_hat = (d * M) + (((1 - d) / N) * np.ones((N, N)))
    for i in range(num_iterations):
        v = M_hat @ v
    return v

pr = pagerank(stochastic_matrix) # Compute the PageRank values

index_to_token = {i: token for i, token in enumerate(voc)} # Create a dictionary to map index to token

sorted_tokens = sorted(index_to_token.items(), key=lambda x: pr[x[0]], reverse=True) # Sort the tokens by their PageRank values

for i in range(10):  # Print the top 10 tokens and their PageRank values
    print(f"{sorted_tokens[i][1]}: {pr[sorted_tokens[i][0]][0]}")

security: 3.1359614855387134e-06
system: 2.3122636217874822e-06
software: 2.039313602769593e-06
information: 1.5352542405737604e-06
testing: 1.0693571188309863e-06
computer: 9.945621279230246e-07
application: 9.685429586986447e-07
datum: 9.362299914780435e-07
code: 9.20537476406246e-07
user: 8.939878351435672e-07


(3) (22 points) Extend the Jupyter notebook (in Python) created in 1) and 2) by providing solutions to the following tasks:
    
(a) (5 points) Create a copy (named e.g. “dice_matrix”) of the co-occurrence matrix obtained in 1c), calculate the significances of the co-occurrences using the Dice coefficient, and update the 
elements in “dice_matrix” accordingly.

In [11]:
dice_matrix = np.copy(coocc_matrix) # Make a copy of the co-occurrence matrix

for i in range(dice_matrix.shape[0]): # Calculate the Dice coefficients
    for j in range(dice_matrix.shape[1]):
       
        numerator = 2 * coocc_matrix[i,j]  # Calculate the numerator and denominator of the Dice coefficient
        denominator = np.sum(coocc_matrix[i,:]) + np.sum(coocc_matrix[:,j])
                
        if denominator != 0: # Calculate the Dice coefficient
            dice_coef = numerator / denominator
        else:
            dice_coef = 0
                
        dice_matrix[i,j] = dice_coef # Update the dice_matrix with the Dice coefficient

print("Co-occurrence matrix:\n", coocc_matrix) # Print the co-occurrence matrix and the Dice coefficient matrix
print("Dice coefficient matrix:\n", dice_matrix)

Co-occurrence matrix:
 [[436. 189. 178. ...   0.   0.   0.]
 [189. 166.  99. ...   0.   0.   0.]
 [178.  99. 176. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   1.   1.]
 [  0.   0.   0. ...   1.   0.   1.]
 [  0.   0.   0. ...   1.   1.   0.]]
Dice coefficient matrix:
 [[0.08141923 0.04271669 0.03825489 ... 0.         0.         0.        ]
 [0.04271669 0.04751002 0.02659503 ... 0.         0.         0.        ]
 [0.03825489 0.02659503 0.04454568 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.25       0.25      ]
 [0.         0.         0.         ... 0.25       0.         0.25      ]
 [0.         0.         0.         ... 0.25       0.25       0.        ]]


(b) (2 points) Output the Dice significance values of the co-occurrences ("software", "security") and ("security", "software"). These must match.

In [12]:
print(voc.index("software"))
print(voc.index("security"))

print(dice_matrix[0][1]) 
print(dice_matrix[1][0])
assert dice_matrix[0][1] == dice_matrix[1][0] # assert will raises an exception if the condition is not met 
                                                # Because this assert is not raising an execption hence “dice_matrix[i][j]” and “dice_matrix[j][i]” have the same values.

print(dice_matrix[voc.index("software")][voc.index("security")]) 
print(dice_matrix[voc.index("security")][voc.index("software")]) 
assert dice_matrix[voc.index("software")][voc.index("security")] == dice_matrix[voc.index("software")][voc.index("security")]

1
0
0.04271669115154255
0.04271669115154255
0.04271669115154255
0.04271669115154255


(c) (5 points) Now calculate the value of the cosine similarity between the context vectors of the tokens "software" and "security". You could do that by relying on Numpy’s functions “np.dot()” and “np.linalg.norm()” or by making use of the function “cosine_similarity()” from “sklearn.metrics.pairwise”. In the latter case, the vectors may have to be reshaped. 

Note: The context vector of a term i can be obtained by accessing the i-th row in “dice_matrix”. For instance, the context vector of the term “software” can be accessed by calling dice_matrix[voc.index("software")] .

In [13]:
software_context = dice_matrix[voc.index("software")] # Accessing the context vectors of "software" and "security"
security_context = dice_matrix[voc.index("security")]

cosine_sim = np.dot(software_context, security_context) / (np.linalg.norm(software_context) * np.linalg.norm(security_context)) # Calculating the cosine similarity between the context vectors

print("Cosine similarity between 'software' and 'security':", cosine_sim)

Cosine similarity between 'software' and 'security': 0.5525394550251929


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

software_context = dice_matrix[voc.index("software")].reshape(1, -1) # Accessing the context vectors of "software" and "security" and reshaping them
security_context = dice_matrix[voc.index("security")].reshape(1, -1)

cosine_sim = cosine_similarity(software_context, security_context)[0][0] # Calculating the cosine similarity between the context vectors

print("Cosine similarity between 'software' and 'security':", cosine_sim)

Cosine similarity between 'software' and 'security': 0.5525394550251929


(d) (3 points) Compare the values obtained in 3b) and 3c). What do you notice? How do you interpret the different values? Formulate complete answer sentences and enter them in the Jupyter 
notebook

Answer: 

Here, question 3(b) computes the Dice significance values of the co-occurrences and shows that for two tokens its same irrespective of order of occurance. Similarly question 3(c) measures the calculate the value of the cosine similarity between the context vectors of the tokens "software" and "security". Dice significance value and cosine similarity are two distinct metrics for comparing the similarity of two collections of data, such as two sets of words in a corpus.

The question 3(b)'s Dice coefficient compares the size of two sets' intersection and union to determine how similar they are. The Dice coefficient in the context of text analysis can be used to compare two documents based on the overlap of their word sets or to compare two words based on their co-occurrence in a corpus.

In contrast, question 3(c)'s cosine similarity calculates the angle between two sets' respective vectors in a multidimensional space to determine how similar they are. The angle between two documents' respective vector representations in a high-dimensional vector space, such as a term frequency-inverse document frequency (TF-IDF) space, is frequently employed in the context of text analysis to quantify how similar two texts are.

Both the Dice coefficient and cosine similarity can be used to measure similarity between words based on their co-occurrence in a corpus. However, the Dice coefficient is based on the size of the intersection and union of the co-occurrence sets, while cosine similarity is based on the angle between the corresponding vector representations. Additionally, the Dice coefficient is a binary measure of similarity, indicating whether two words co-occur or not, while cosine similarity is a continuous measure of similarity, indicating the degree of similarity between the two vectors.

(e) (7 points) What are the 10 most similar words/tokens to “software” in descending order of their cosine similarity scores? For this purpose, calculate the cosine similarity values between the context vector of the token “software” and all row vectors in “dice_matrix”. Hint: The function "cosine_similarity()" from "sklearn.metrics.pairwise" can be passed this matrix as a single 
parameter. Note: the most similar token to “software” must be “software”.

In [15]:
software_context = dice_matrix[voc.index("software")].reshape(1, -1) # Accessing the context vector of "software" and reshaping it

similarity_scores = cosine_similarity(software_context, dice_matrix) # Calculating the cosine similarity between the context vector of "software" and all row vectors in "dice_matrix"

top10_similar_words = [voc[i] for i in np.argsort(similarity_scores[0])[::-1][:10]] # Sorting the scores in descending order and retrieving the top 10 words/tokens

print("The 10 most similar words/tokens to 'software' are:")
for i, word in enumerate(top10_similar_words):
    print(i+1, "-", word)

The 10 most similar words/tokens to 'software' are:
1 - software
2 - process
3 - development
4 - testing
5 - system
6 - security
7 - product
8 - requirement
9 - quality
10 - program
